In [2]:
%load_ext dotenv
%dotenv

In [1]:
! dvc pull

eatures/train.pkl    440k/5.41M [00:00<01:12,    72.0kB/s]

 39%|███▉      |data/prepared/test.tsv    1.78M/4.54M [00:00<00:25,     115kB/s]
 24%|██▍       |data/features/test.pkl     328k/1.35M [00:01<00:26,    41.0kB/s]


  2%|▏         |data/data.xml              576k/36.1M [00:00<08:46,    70.9kB/s]




 12%|█▏        |data/features/train.pkl    656k/5.41M [00:00<00:49,     101kB/s]





  2%|▏         |data/prepared/train.tsv    368k/18.2M [00:00<04:11,    74.3kB/s]
 33%|███▎      |data/features/test.pkl     456k/1.35M [00:01<00:16,    57.6kB/s]



  5%|▍         |model.pkl                  128k/2.54M [00:01<01:25,    29.5kB/s]

 49%|████▉     |data/prepared/test.tsv    2.22M/4.54M [00:01<00:15,     162kB/s]


  2%|▏         |data/data.xml              704k/36.1M [00:01<06:17,    98.4kB/s]




 15%|█▍        |data/features/train.pkl    816k/5.41M [00:01<00:34,     141kB/s]





  3%|▎         |data/prepared/train.tsv    472k/18.2M [00:01<03:00,     103kB/s]



 13%|█▎        |mode

## Easily reproduce somewhere else

### from cache

In [3]:
! pipenv run dvc repro

Loading .env environment variables...
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Stage 'data/data.xml.dvc' didn't change, skipping
Stage 'prepare' didn't change, skipping
Stage 'featurize' didn't change, skipping
Stage 'train' didn't change, skipping
Stage 'evaluate' didn't change, skipping
Data and pipelines are up to date.


### or force actual pipeline run

In [4]:
! dvc repro -f

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Verifying data sources in stage: 'data/data.xml.dvc'

Running stage 'prepare' with command:
	python src/prepare.py data/data.xml
  0% Saving prepared|                          |0.00/2.00 [00:00<?,     ?file/s]
Running stage 'featurize' with command:
	python src/featurization.py data/prepared data/features
The input data frame data/prepared/train.tsv size is (20017, 3)
The output matrix data/features/train.pkl size is (20017, 502) and data type is float64
The input data frame data/prepared/test.tsv size is (4983, 3)
The output matrix data/features/test.pkl size is (4983, 502) and data type is float64
  0% Saving features|                          |0.00/2.00 [00:00<?,     ?file/s]
Running stage 'train' with command:
	python src/train.py data/features model.pkl
Input matrix size (20017, 502)
X matrix size (20017, 500)
Y matrix size (20017,)
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>
Running s

In [5]:
! dvc diff

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>

### Assert equivalent result

In [6]:
! cat scores.json

{"auc": 0.5417487597055675}

### Now I can easily
* Making updates to the models
* see differences in parameters or results
* collaborate with others
* reproduce results

### Go and modify some parameters

In [8]:
! dvc status

featurize:
	changed deps:
		params.yaml:
			modified:           featurize.max_features
train:
	changed deps:
		params.yaml:
			modified:           train.n_estimators


In [6]:
! dvc params diff

Path         Param                   Old    New
params.yaml  featurize.max_features  500    1500
params.yaml  train.n_estimators      50     100


In [9]:
! dvc repro

If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Stage 'data/data.xml.dvc' didn't change, skipping
Stage 'prepare' didn't change, skipping
Running stage 'featurize' with command:
	python src/featurization.py data/prepared data/features
The input data frame data/prepared/train.tsv size is (20017, 3)
The output matrix data/features/train.pkl size is (20017, 1502) and data type is float64
The input data frame data/prepared/test.tsv size is (4983, 3)
The output matrix data/features/test.pkl size is (4983, 1502) and data type is float64
  0% Saving features|                          |0.00/2.00 [00:00<?,     ?file/s]Updating lock file 'dvc.lock'

Running stage 'train' with command:
	python src/train.py data/features model.pkl
Input matrix size (20017, 1502)
X matrix size (20017, 1500)
Y matrix size (20017,)
If DVC froze, see `hardlink_lock` in <https://man.dvc.org/config#core>Updating lock file 'dvc.lock'

Running stage 'evaluate' with command:
	python src/evaluate.py mo

In [10]:
! dvc metrics diff

Path         Metric    Value    Change
scores.json  auc       0.56925  0.0275


In [13]:
! cat scores.json

{"auc": 0.5692532520676357}

In [12]:
! dvc plots diff -x recall -y precision

file:///home/nichita/Projects/dvc_intro/plots.html
